### 1. **Transformações Básicas**

- **`Map`**: Aplica uma função a cada elemento do PCollection. Ideal para transformações simples.
  
  ```python
  | 'Transformar' >> beam.Map(lambda x: x * 2)
  ```

- **`Filter`**: Filtra elementos com base em uma condição. Útil para remover elementos indesejados.

  ```python
  | 'Filtrar' >> beam.Filter(lambda x: x > 10)
  ```

- **`FlatMap`**: Semelhante ao `Map`, mas permite que a função retorne uma lista de elementos, "achatando" a saída.

  ```python
  | 'Dividir' >> beam.FlatMap(lambda x: x.split())
  ```

### 2. **Normalizações**

- **`Map`**: Pode ser usado para normalizar dados, como converter strings para minúsculas.

  ```python
  | 'Normalizar' >> beam.Map(lambda x: x.lower())
  ```

- **`Reduce`**: Usado para agregar dados. Pode ser utilizado para calcular médias, somas, etc.

  ```python
  | 'Somar' >> beam.CombineGlobally(sum)
  ```

### 3. **Agrupamentos**

- **`GroupByKey`**: Agrupa elementos por chave, permitindo que você agregue ou processe dados em grupos.

  ```python
  | 'Agrupar' >> beam.GroupByKey()
  ```

- **`CoGroupByKey`**: Similar ao `GroupByKey`, mas agrupa dados de dois ou mais PCollections por chave.

### 4. **Substituições de Caracteres**

- **`Map`**: Você pode usar expressões regulares ou funções de string para substituir caracteres em strings.

  ```python
  import re
  
  | 'Substituir' >> beam.Map(lambda x: re.sub(r'\W+', '', x))
  ```

### 5. **Uniões e Interseções**

- **`Union`**: Combina múltiplos PCollections em um único PCollection.

  ```python
  combined = (pcollection1, pcollection2) | 'Unir' >> beam.Flatten()
  ```

- **`Intersection`**: Embora não haja uma função nativa, você pode obter a interseção utilizando `GroupByKey` seguido de `Filter`.

### 6. **Divisão de Dados**

- **`Partition`**: Divide um PCollection em múltiplos PCollections com base em uma função de partição.

  ```python
  def partition_fn(element, num_partitions):
      return 0 if element < 10 else 1
  
  | 'Dividir' >> beam.Partition(partition_fn, 2)
  ```

### 7. **Combinação e Agregação**

- **`CombineGlobally`**: Aplica uma função de combinação a todos os elementos do PCollection.

  ```python
  | 'Combinar' >> beam.CombineGlobally(sum)
  ```

- **`Top`**: Retorna os N maiores elementos de um PCollection.

  ```python
  | 'Top N' >> beam.combiners.Top.Largest(5)
  ```

### 8. **Gravação e Leitura de Dados**

- **`ReadFromText`** e **`WriteToText`**: Leitura e escrita de arquivos de texto.

- **`ReadFromBigQuery`** e **`WriteToBigQuery`**: Integração com Google BigQuery.

### Conclusão

Essas são algumas das funções mais comuns no Apache Beam para manipulação de dados em pipelines. Você pode combiná-las de várias maneiras para atender às suas necessidades específicas de processamento de dados.

Se você quiser mais detalhes sobre alguma função ou exemplo específico, fique à vontade para perguntar!

In [ ]:
import apache_beam as beam
from datetime import datetime
import logging

# Configurando o logging
logging.basicConfig(level=logging.INFO)

def transform_data(row):
    # Extrai nome e idade da linha
    nome, idade = row
    ano_atual = datetime.now().year
    ano_nascimento = ano_atual - int(idade)
    
    # Logando a transformação
    logging.info(f'Transformando: {row} -> Ano de Nascimento: {ano_nascimento}')
    
    return {
        'nome': nome,
        'ano_nascimento': ano_nascimento
    }

# Definindo o pipeline
with beam.Pipeline() as pipeline:
    (
        # Lê o arquivo CSV
        pipeline
        | 'Ler CSV' >> beam.io.ReadFromText('input.csv')
        
        # Divide cada linha em uma lista
        | 'Dividir Linhas' >> beam.Map(lambda line: line.split(','))
        
        # Remove a linha de cabeçalho
        | 'Remover Cabeçalho' >> beam.Filter(lambda line: line[0] != 'nome')
        
        # Aplica a transformação para calcular o ano de nascimento
        | 'Transformar Dados' >> beam.Map(transform_data)
        
        # Formata os dados para CSV
        | 'Formatar para CSV' >> beam.Map(lambda record: f"{record['nome']},{record['ano_nascimento']}")
        
        # Escreve o resultado em um novo arquivo CSV
        | 'Escrever CSV' >> beam.io.WriteToText('output', file_name_suffix='.csv')
    )


Sim, você pode obter uma prévia dos dados carregados inicialmente no Apache Beam, embora não haja uma função específica para isso, como um "preview". No entanto, você pode usar a função `Map` para imprimir os dados ou coletar uma amostra antes de prosseguir com as transformações. Aqui estão algumas abordagens que você pode considerar:

### 1. Usando `Map` para Imprimir os Dados

Uma maneira simples de ver os dados carregados é adicionar um passo no seu pipeline para imprimir os elementos. Aqui está um exemplo:

```python
import apache_beam as beam

# Definindo o pipeline
with beam.Pipeline() as pipeline:
    (
        pipeline
        | 'Ler CSV' >> beam.io.ReadFromText('input.csv')
        | 'Dividir Linhas' >> beam.Map(lambda line: line.split(','))
        | 'Imprimir Dados Iniciais' >> beam.Map(print)  # Exibir os dados iniciais
        | 'Transformar Dados' >> beam.Map(transform_data)
        | 'Escrever CSV' >> beam.io.WriteToText('output', file_name_suffix='.csv')
    )
```

### 2. Usando `Take`

Se você não quiser imprimir todos os dados, pode usar o método `Take` para obter uma amostra limitada de elementos. Assim, você pode ver uma quantidade controlada de dados:

```python
import apache_beam as beam

with beam.Pipeline() as pipeline:
    dados_iniciais = (
        pipeline
        | 'Ler CSV' >> beam.io.ReadFromText('input.csv')
        | 'Dividir Linhas' >> beam.Map(lambda line: line.split(','))
        | 'Amostra' >> beam.combiners.Sample.FixedSizeGlobally(5)  # Pega 5 elementos
    )
    
    # Usar 'Map' para imprimir a amostra
    dados_iniciais | 'Imprimir Amostra' >> beam.Map(print)

    # Continuar com o restante do pipeline
    # ...
```

### 3. Usando `PCollection` para Coletar Dados

Se você estiver trabalhando em um ambiente de desenvolvimento local e quiser coletar dados em uma lista, você pode usar o método `to_list()`:

```python
with beam.Pipeline() as pipeline:
    dados_iniciais = (
        pipeline
        | 'Ler CSV' >> beam.io.ReadFromText('input.csv')
        | 'Dividir Linhas' >> beam.Map(lambda line: line.split(','))
    )

    # Coletar os dados para uma lista
    dados_coletados = dados_iniciais | beam.combiners.ToList()

    # Usar 'Map' para imprimir os dados coletados
    dados_coletados | 'Imprimir Dados' >> beam.Map(print)
```

### Observações

- **Impressão no Console**: Quando você usa `print`, lembre-se de que a saída pode não ser exibida em tempo real, dependendo do runner que você está usando. No `DirectRunner`, você verá a saída imediatamente.
- **Performance**: Coletar todos os dados em memória pode não ser ideal para grandes volumes de dados. Para grandes datasets, recomenda-se usar métodos de amostragem.

Essas abordagens devem ajudar você a obter uma visão inicial dos dados carregados em seu pipeline. Se precisar de mais detalhes ou exemplos, fique à vontade para perguntar!